In [1]:
import pandas as pd

In [2]:
import json
import sqlite3
from io import StringIO
from itertools import islice

In [3]:
# peek the data
chunks = pd.read_json('arxiv-data.json', lines=True, chunksize=5)
peek = next(islice(chunks,0,1))
peek

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,..."
1,704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]"
2,704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,The evolution of Earth-Moon system is descri...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13,"[[Pan, Hongjun, ]]"
3,704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,None,math.CO,None,We show that a determinant of Stirling cycle...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23,"[[Callan, David, ]]"
4,704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,math.CA math.FA,None,In this paper we show how to compute the $\L...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2013-10-15,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]"


In [31]:
# initialize a database
keys = peek.columns.tolist()
keys_converted = ['"'+key+'"' for key in keys]
str_keys = '(' + ','.join(keys_converted) + ')'
# str_vals = '(' + ','.join(['?']*len(keys)) + ')'
print(str_keys)
conn = sqlite3.connect('arxiv.db')
cursor = conn.cursor()
cursor.execute('DROP TABLE IF EXISTS arxiv')
cursor.execute('CREATE TABLE arxiv'+str_keys)

("id","submitter","authors","title","comments","journal-ref","doi","report-no","categories","license","abstract","versions","update_date","authors_parsed")


In [32]:
keys

['id',
 'submitter',
 'authors',
 'title',
 'comments',
 'journal-ref',
 'doi',
 'report-no',
 'categories',
 'license',
 'abstract',
 'versions',
 'update_date',
 'authors_parsed']

In [38]:
# migrate data to database
line = '_'

with open('arxiv-data.json', 'r') as f:
    while line:
        line = f.readline()
        line = json.loads(line)
        # insert_sql = 'INSERT INTO arxiv' + str_keys + ' VALUES ' + str_vals
        # cursor.executemany(insert_sql, insert_sql)
        print(line)
        vals = ['"'+str(line[key])+'"' for key in keys]
        str_vals = '('+','.join(vals)+')'
        print(str_vals)
        cursor.execute('INSERT INTO arxiv VALUES {}'.format(str_vals))
        conn.commit()
        break

cursor.close()
conn.close()

{'id': '0704.0001', 'submitter': 'Pavel Nadolsky', 'authors': "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan", 'title': 'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies', 'comments': '37 pages, 15 figures; published version', 'journal-ref': 'Phys.Rev.D76:013009,2007', 'doi': '10.1103/PhysRevD.76.013009', 'report-no': 'ANL-HEP-PR-07-12', 'categories': 'hep-ph', 'license': None, 'abstract': '  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab

In [34]:
[(2,3)]

[(2, 3)]

In [39]:
conn = sqlite3.connect('arxiv.db')
cursor = conn.cursor()
cursor.execute('SELECT * FROM arxiv')
cursor.fetchall()

[('0704.0001',
  'Pavel Nadolsky',
  "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan",
  'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies',
  '37 pages, 15 figures; published version',
  'Phys.Rev.D76:013009,2007',
  '10.1103/PhysRevD.76.013009',
  'ANL-HEP-PR-07-12',
  'hep-ph',
  'None',
  '  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\nmore detailed tests with CDF and DO data. Predictions are s

In [41]:
cursor.execute('SELECT id FROM arxiv')
cursor.fetchall()

[('0704.0001',)]